In [1]:
import yfinance as yf
import pandas as pd
import pickle
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import seaborn as sns
from functools import reduce
import numpy as np

In [154]:
DOW_30_TICKERS = [
    "AXP",
    "AMGN",
    "AAPL",
    "BA",
    "CAT",
    "CSCO",
    "CVX",
    "GS",
    "HD",
    "HON",
    "IBM",
    "INTC", 
    "JNJ",
    "KO",
    "JPM",
    "MCD",
    "MMM",
    "MRK",
    "MSFT",
    "NKE",
    "PG",
    "TRV",
    "UNH",
    "CRM",
    "VZ",
    "V",
    "WBA",
    "WMT",
    "DIS",
]

In [155]:
tickers = ' '.join(ticker for ticker in DOW_30_TICKERS)
df = yf.download(tickers, period='2y')
df = df.reset_index().melt('Date')
df = df.rename(columns={'variable_0': 'values', 'variable_1': 'ticker'})
df = df.pivot(index = ['Date', 'ticker'], columns = 'values', values='value').reset_index()
df = df.dropna()
df = pd.DataFrame(df.to_dict())

[*********************100%***********************]  29 of 29 completed


In [156]:
# assign a column order for our dataframes
column_order = ['date', 'open', 'high', 'low', 'close', 'adjcp', 'volume',	'tic', 'day']

# make the index a datetime object
df = df.rename({'Date': 'date', 'ticker': 'tic', 'Adj Close': 'adjcp'}, axis = 1)
df['date'] = pd.to_datetime(df['date'])
# add a column for 'day of the week'
df['day'] = df['date'].dt.dayofweek
# set a column order for our dataframes
df.columns = df.columns.str.lower()
df = df[column_order]
# convert date column back to str
df['date'] = df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [157]:
df

,date,open,high,low,close,adjcp,volume,tic,day
0,2020-04-23,68.967499,70.437500,68.717499,68.757500,67.825500,124814400.0,AAPL,3
1,2020-04-23,229.089996,235.649994,228.880005,232.490005,219.126663,3226400.0,AMGN,3
2,2020-04-23,83.239998,84.779999,81.709999,82.459999,80.373672,5807200.0,AXP,3
3,2020-04-23,136.500000,142.380005,136.050003,137.740005,137.740005,25936400.0,BA,3
4,2020-04-23,111.330002,115.230003,110.949997,112.910004,108.065819,3545400.0,CAT,3
...,...,...,...,...,...,...,...,...,...
14640,2022-04-22,532.059998,533.349976,520.179993,520.940002,520.940002,3452200.0,UNH,4
14641,2022-04-22,216.240005,216.559998,207.949997,208.169998,208.169998,6199000.0,V,4
14642,2022-04-22,53.439999,53.540001,51.459999,51.910000,51.910000,44949500.0,VZ,4
14643,2022-04-22,46.619999,46.799999,45.500000,45.540001,45.540001,5952700.0,WBA,4


In [158]:
from finrl.apps import config_tickers
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.drl_agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts
from finrl.finrl_meta.data_processor import DataProcessor
from finrl.finrl_meta.data_processors.processor_yahoofinance import YahooFinanceProcessor

In [159]:
processed_df = FeatureEngineer(use_technical_indicator = True, use_turbulence = True).preprocess_data(df)

C:\Users\Damola\anaconda3\envs\learn-env\lib\site-packages\finrl\finrl_meta\preprocessor\preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
C:\Users\Damola\anaconda3\envs\learn-env\lib\site-packages\finrl\finrl_meta\preprocessor\preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
C:\Users\Damola\anaconda3\envs\learn-env\lib\site-packages\finrl\finrl_meta\preprocessor\preprocessors.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  indicator_df = indicator_df.append(
C:\Users\Damola\anaconda3\envs\learn-env\lib\site-packages\finrl\finrl_meta\preprocessor\preprocessors.py:154: FutureWarning: The frame.append method is d

Successfully added technical indicators
Successfully added turbulence index


In [160]:
processed_df

,date,open,high,low,close,adjcp,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2020-04-23,68.967499,70.437500,68.717499,68.757500,67.825500,124814400.0,AAPL,3,0.000000,72.557215,66.942785,100.000000,66.666667,100.000000,68.757500,68.757500,0.000000
1,2020-04-23,229.089996,235.649994,228.880005,232.490005,219.126663,3226400.0,AMGN,3,0.000000,72.557215,66.942785,100.000000,66.666667,100.000000,232.490005,232.490005,0.000000
2,2020-04-23,83.239998,84.779999,81.709999,82.459999,80.373672,5807200.0,AXP,3,0.000000,72.557215,66.942785,100.000000,66.666667,100.000000,82.459999,82.459999,0.000000
3,2020-04-23,136.500000,142.380005,136.050003,137.740005,137.740005,25936400.0,BA,3,0.000000,72.557215,66.942785,100.000000,66.666667,100.000000,137.740005,137.740005,0.000000
4,2020-04-23,111.330002,115.230003,110.949997,112.910004,108.065819,3545400.0,CAT,3,0.000000,72.557215,66.942785,100.000000,66.666667,100.000000,112.910004,112.910004,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14640,2022-04-22,532.059998,533.349976,520.179993,520.940002,520.940002,3452200.0,UNH,4,9.464022,552.342664,501.849355,54.620841,28.618750,15.256733,518.389675,498.279838,99.985197
14641,2022-04-22,216.240005,216.559998,207.949997,208.169998,208.169998,6199000.0,V,4,-0.691998,229.895279,206.990716,45.761880,-69.512052,11.885463,216.132331,217.209999,99.985197
14642,2022-04-22,53.439999,53.540001,51.459999,51.910000,51.910000,44949500.0,VZ,4,0.336969,55.271264,50.430735,46.607873,-14.854286,9.976125,52.501333,52.983833,99.985197
14643,2022-04-22,46.619999,46.799999,45.500000,45.540001,45.540001,5952700.0,WBA,4,-0.303541,48.078826,42.622174,46.165745,-7.284241,21.265098,46.044000,46.976834,99.985197


In [161]:
from ta.volatility import BollingerBands
from ta.trend import SMAIndicator, MACD, CCIIndicator, EMAIndicator, IchimokuIndicator, AroonIndicator
from ta.momentum import RSIIndicator, stoch, WilliamsRIndicator
from ta.volume import OnBalanceVolumeIndicator, MFIIndicator

In [162]:
#Log returns
processed_df['log_return'] = np.log(processed_df['adjcp'] / processed_df['adjcp'].shift(1))
# CCI
_cci = CCIIndicator(high = processed_df['high'],
                    low = processed_df['low'], 
                    close = processed_df['close'],
                    window = 20)
processed_df['20_day_CCI'] = _cci.cci()
# Ichimoku Indicator
ichi = IchimokuIndicator(high = processed_df['high'],
                            low = processed_df['low'])
                            
processed_df['ichimoku_span_a'] = ichi.ichimoku_a()
processed_df['ichimoku_span_b'] = ichi.ichimoku_b()
processed_df['ichimoku_span_baseline'] = ichi.ichimoku_base_line()
processed_df['ichimoku_span_conversion_line'] = ichi.ichimoku_conversion_line()
# ArronIndicator
_aroon = AroonIndicator(close = processed_df['close'])
processed_df['Aroon_down'] = _aroon.aroon_down()
processed_df['Aroon_up'] = _aroon.aroon_up()
processed_df['Aroon_indicator'] = _aroon.aroon_indicator()
#Williams R Indicator
processed_df['wiilliams_r'] = WilliamsRIndicator(high = processed_df['high'],
                                        low = processed_df['low'],
                                        close = processed_df['close']
                                        ).williams_r()
# On Balance Volume
processed_df['on_balance_volume'] = OnBalanceVolumeIndicator(close = processed_df['close'],
                                                    volume = processed_df['volume']
                                                    ).on_balance_volume()
#MFI
processed_df['mfi'] = MFIIndicator(high = processed_df['high'],
                        low = processed_df['low'],
                        close = processed_df['close'],
                        volume = processed_df['volume']
                        ).money_flow_index()

In [163]:
DJIA = processed_df
DJIA

,date,open,high,low,close,adjcp,volume,tic,day,macd,...,ichimoku_span_a,ichimoku_span_b,ichimoku_span_baseline,ichimoku_span_conversion_line,Aroon_down,Aroon_up,Aroon_indicator,wiilliams_r,on_balance_volume,mfi
0,2020-04-23,68.967499,70.437500,68.717499,68.757500,67.825500,124814400.0,AAPL,3,0.000000,...,NaN,69.577499,NaN,NaN,NaN,NaN,NaN,NaN,1.248144e+08,NaN
1,2020-04-23,229.089996,235.649994,228.880005,232.490005,219.126663,3226400.0,AMGN,3,0.000000,...,NaN,152.183746,NaN,NaN,NaN,NaN,NaN,NaN,1.280408e+08,NaN
2,2020-04-23,83.239998,84.779999,81.709999,82.459999,80.373672,5807200.0,AXP,3,0.000000,...,NaN,152.183746,NaN,NaN,NaN,NaN,NaN,NaN,1.222336e+08,NaN
3,2020-04-23,136.500000,142.380005,136.050003,137.740005,137.740005,25936400.0,BA,3,0.000000,...,NaN,152.183746,NaN,NaN,NaN,NaN,NaN,NaN,1.481700e+08,NaN
4,2020-04-23,111.330002,115.230003,110.949997,112.910004,108.065819,3545400.0,CAT,3,0.000000,...,NaN,152.183746,NaN,NaN,NaN,NaN,NaN,NaN,1.446246e+08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14640,2022-04-22,532.059998,533.349976,520.179993,520.940002,520.940002,3452200.0,UNH,4,9.464022,...,294.594988,297.115000,289.924988,299.264988,56.0,100.0,44.0,-2.549034,-4.547335e+10,64.219431
14641,2022-04-22,216.240005,216.559998,207.949997,208.169998,208.169998,6199000.0,V,4,-0.691998,...,299.424988,297.115000,289.924988,308.924988,52.0,96.0,44.0,-66.792645,-4.547955e+10,62.134303
14642,2022-04-22,53.439999,53.540001,51.459999,51.910000,51.910000,44949500.0,VZ,4,0.336969,...,291.164988,297.115000,289.924988,292.404987,48.0,92.0,44.0,-98.888775,-4.552450e+10,58.343908
14643,2022-04-22,46.619999,46.799999,45.500000,45.540001,45.540001,5952700.0,WBA,4,-0.303541,...,289.424988,296.750000,289.424988,289.424988,100.0,88.0,-12.0,-99.991801,-4.553045e+10,61.040486


In [164]:
DJIA = DJIA.sort_values(['date', 'tic'], ignore_index=True)
DJIA.index = DJIA.date.factorize()[0]

cov_list = []
return_list = []

lookback = 252
for i in range(lookback, len(DJIA.index.unique())):
    data_lookback = DJIA.loc[i - lookback: i, :]
    price_lookback = data_lookback.pivot_table(index = 'date', columns = 'tic', values = 'close')
    
    return_lookback = price_lookback.pct_change().dropna()
    return_list.append(return_lookback)

    covs = return_lookback.cov().values
    cov_list.append(covs)

DJIA_cov = pd.DataFrame({'date': DJIA.date.unique()[lookback:], 'cov_list': cov_list, 'return_list': return_list})

In [165]:
DJIA

,date,open,high,low,close,adjcp,volume,tic,day,macd,...,ichimoku_span_a,ichimoku_span_b,ichimoku_span_baseline,ichimoku_span_conversion_line,Aroon_down,Aroon_up,Aroon_indicator,wiilliams_r,on_balance_volume,mfi
0,2020-04-23,68.967499,70.437500,68.717499,68.757500,67.825500,124814400.0,AAPL,3,0.000000,...,NaN,69.577499,NaN,NaN,NaN,NaN,NaN,NaN,1.248144e+08,NaN
0,2020-04-23,229.089996,235.649994,228.880005,232.490005,219.126663,3226400.0,AMGN,3,0.000000,...,NaN,152.183746,NaN,NaN,NaN,NaN,NaN,NaN,1.280408e+08,NaN
0,2020-04-23,83.239998,84.779999,81.709999,82.459999,80.373672,5807200.0,AXP,3,0.000000,...,NaN,152.183746,NaN,NaN,NaN,NaN,NaN,NaN,1.222336e+08,NaN
0,2020-04-23,136.500000,142.380005,136.050003,137.740005,137.740005,25936400.0,BA,3,0.000000,...,NaN,152.183746,NaN,NaN,NaN,NaN,NaN,NaN,1.481700e+08,NaN
0,2020-04-23,111.330002,115.230003,110.949997,112.910004,108.065819,3545400.0,CAT,3,0.000000,...,NaN,152.183746,NaN,NaN,NaN,NaN,NaN,NaN,1.446246e+08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,2022-04-22,532.059998,533.349976,520.179993,520.940002,520.940002,3452200.0,UNH,4,9.464022,...,294.594988,297.115000,289.924988,299.264988,56.0,100.0,44.0,-2.549034,-4.547335e+10,64.219431
504,2022-04-22,216.240005,216.559998,207.949997,208.169998,208.169998,6199000.0,V,4,-0.691998,...,299.424988,297.115000,289.924988,308.924988,52.0,96.0,44.0,-66.792645,-4.547955e+10,62.134303
504,2022-04-22,53.439999,53.540001,51.459999,51.910000,51.910000,44949500.0,VZ,4,0.336969,...,291.164988,297.115000,289.924988,292.404987,48.0,92.0,44.0,-98.888775,-4.552450e+10,58.343908
504,2022-04-22,46.619999,46.799999,45.500000,45.540001,45.540001,5952700.0,WBA,4,-0.303541,...,289.424988,296.750000,289.424988,289.424988,100.0,88.0,-12.0,-99.991801,-4.553045e+10,61.040486


In [166]:
DJIA = DJIA.merge(DJIA_cov, on = 'date')
DJIA = DJIA.sort_values(['date', 'tic']).reset_index(drop = True)

In [167]:
DJIA

,date,open,high,low,close,adjcp,volume,tic,day,macd,...,ichimoku_span_baseline,ichimoku_span_conversion_line,Aroon_down,Aroon_up,Aroon_indicator,wiilliams_r,on_balance_volume,mfi,cov_list,return_list
0,2021-04-23,132.160004,135.119995,132.160004,134.320007,133.526398,78657500.0,AAPL,4,2.555536,...,225.070005,225.640005,8.0,80.0,72.0,-76.374768,-4.240727e+10,38.122099,"[[0.0004925018826583311, 0.0001120364210100210...",tic AAPL AMGN AXP ...
1,2021-04-23,254.979996,258.140015,253.669998,257.029999,249.035507,2426900.0,AMGN,4,4.153103,...,225.070005,225.640005,4.0,76.0,72.0,-40.934036,-4.240484e+10,40.529768,"[[0.0004925018826583311, 0.0001120364210100210...",tic AAPL AMGN AXP ...
2,2021-04-23,141.539993,144.979996,140.679993,144.330002,142.832474,6395300.0,AXP,4,1.473194,...,225.070005,225.640005,84.0,72.0,-12.0,-73.483711,-4.241123e+10,38.236919,"[[0.0004925018826583311, 0.0001120364210100210...",tic AAPL AMGN AXP ...
3,2021-04-23,233.889999,238.580002,233.110001,238.380005,238.380005,11533100.0,BA,4,-1.165399,...,225.640005,225.640005,80.0,68.0,-12.0,-46.320472,-4.239970e+10,44.507743,"[[0.0004925018826583311, 0.0001120364210100210...",tic AAPL AMGN AXP ...
4,2021-04-23,229.360001,231.649994,227.880005,230.110001,225.444550,3458300.0,CAT,4,2.177335,...,225.640005,155.330008,76.0,64.0,-12.0,-48.708989,-4.240316e+10,44.809914,"[[0.0004925018826583311, 0.0001120364210100210...",tic AAPL AMGN AXP ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7332,2022-04-22,532.059998,533.349976,520.179993,520.940002,520.940002,3452200.0,UNH,4,9.464022,...,289.924988,299.264988,56.0,100.0,44.0,-2.549034,-4.547335e+10,64.219431,"[[0.00023816430537387597, 5.4008687924133486e-...",tic AAPL AMGN AXP ...
7333,2022-04-22,216.240005,216.559998,207.949997,208.169998,208.169998,6199000.0,V,4,-0.691998,...,289.924988,308.924988,52.0,96.0,44.0,-66.792645,-4.547955e+10,62.134303,"[[0.00023816430537387597, 5.4008687924133486e-...",tic AAPL AMGN AXP ...
7334,2022-04-22,53.439999,53.540001,51.459999,51.910000,51.910000,44949500.0,VZ,4,0.336969,...,289.924988,292.404987,48.0,92.0,44.0,-98.888775,-4.552450e+10,58.343908,"[[0.00023816430537387597, 5.4008687924133486e-...",tic AAPL AMGN AXP ...
7335,2022-04-22,46.619999,46.799999,45.500000,45.540001,45.540001,5952700.0,WBA,4,-0.303541,...,289.424988,289.424988,100.0,88.0,-12.0,-99.991801,-4.553045e+10,61.040486,"[[0.00023816430537387597, 5.4008687924133486e-...",tic AAPL AMGN AXP ...


In [168]:
from stable_baselines3.common.vec_env import DummyVecEnv
import gym
from gym import spaces

class StockPortfolioEnv(gym.Env):
    """A single stock trading environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then 
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step
        

    """
    metadata = {'render.modes': ['human']}

    def __init__(self, 
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                turbulence_threshold=None,
                lookback=252,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,)) 
        # Shape = (34, 30)
        # covariance matrix + technical indicators
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False     
        self.turbulence_threshold = turbulence_threshold        
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

        
    def step(self, actions):
        # print(self.day)
        self.terminal = self.day >= len(self.df.index.unique())-1
        # print(actions)

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(),'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()
            
            plt.plot(self.portfolio_return_memory,'r')
            plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))           
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() !=0:
              sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                       df_daily_return['daily_return'].std()
              print("Sharpe: ",sharpe)
            print("=================================")
            
            return self.state, self.reward, self.terminal,{}

        else:
            #print("Model actions: ",actions)
            # actions are the portfolio weight
            # normalize to sum of 1
            #if (np.array(actions) - np.array(actions).min()).sum() != 0:
            #  norm_actions = (np.array(actions) - np.array(actions).min()) / (np.array(actions) - np.array(actions).min()).sum()
            #else:
            #  norm_actions = actions
            weights = self.softmax_normalization(actions) 
            #print("Normalized actions: ", weights)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
            #print(self.state)
            # calcualte portfolio return
            # individual stocks' return * weight
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values)-1)*weights)
            # update portfolio value
            new_portfolio_value = self.portfolio_value*(1+portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])            
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value 
            #print("Step reward: ", self.reward)
            #self.reward = self.reward*self.reward_scaling

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False 
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]] 
        return self.state
    
    def render(self, mode='human'):
        return self.state
        
    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output

    
    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']
        
        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [186]:
DJIA_dates = list(DJIA['date'])
DJIA_dates = sorted(set(DJIA_dates), key=DJIA_dates.index)
DJIA_start = DJIA_dates[-2]

In [187]:
from datetime import date
today = date.today()

In [188]:
test = data_split(DJIA, DJIA_start, str(today))

In [190]:
# observation space
stock_dimension = len(test['tic'].unique())
state_space = stock_dimension
print(f'Stock Dimension: {stock_dimension}, State_space: {state_space}')

Stock Dimension: 29, State_space: 29


In [191]:
technical_indicators_list = list(DJIA.columns[9:-2])

env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "transaction_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": technical_indicators_list, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
}

In [192]:
e_trade_gym = StockPortfolioEnv(df = test, **env_kwargs)

In [206]:
from stable_baselines3 import A2C

In [225]:
model = A2C.load('trained_models/untuned_A2C')

In [226]:
df_daily_return, df_actions = DRLAgent.DRL_prediction(model = model, 
                                                      environment= e_trade_gym)

begin_total_asset:1000000
end_total_asset:972435.7463813948
Sharpe:  -11.224972160321824
hit end!


In [227]:
df_daily_return

,date,daily_return
0,2022-04-21,0.000000
1,2022-04-22,-0.027564


In [228]:
df_actions

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,GS,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
date,,,,,,,,,,,,,,,,,,,,,
2022-04-21,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,...,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483
2022-04-22,0.038677,0.030392,0.030392,0.046047,0.030392,0.039858,0.030392,0.030476,0.030392,0.044069,...,0.030392,0.030392,0.032950,0.032850,0.030392,0.059186,0.030392,0.033766,0.030392,0.035613


In [282]:
new_df = df_actions.T
new_df.columns = ['initial_holdings', 'recommended_holdings']
new_df['holding_pct_change'] = round(((100/new_df['initial_holdings']) * new_df['recommended_holdings']) - 100, 3)
new_df['initial_holdings'] = round(new_df['initial_holdings'], 3)
new_df['recommended_holdings'] = round(new_df['recommended_holdings'], 3)
new_df = new_df.reset_index()
new_df = new_df.rename({'index': 'ticker'}, axis = 1)

In [285]:
new_df

,ticker,initial_holdings,recommended_holdings,holding_pct_change
0,AAPL,0.034,0.039,12.163
1,AMGN,0.034,0.030,-11.863
2,AXP,0.034,0.030,-11.863
3,BA,0.034,0.046,33.537
4,CAT,0.034,0.030,-11.863
5,CRM,0.034,0.040,15.588
6,CSCO,0.034,0.030,-11.863
7,CVX,0.034,0.030,-11.620
8,DIS,0.034,0.030,-11.863
9,GS,0.034,0.044,27.801


In [283]:
data_list = []

for ticker in new_df['ticker']:
    df = new_df[new_df['ticker'] == ticker]
    data_list.append([
            list(df['ticker'])[0], 
            list(df['initial_holdings'])[0], 
            list(df['recommended_holdings'])[0], 
            list(df['holding_pct_change'])[0]
        ])

In [284]:
data_list

[['AAPL', 0.034, 0.039, 12.163],
 ['AMGN', 0.034, 0.03, -11.863],
 ['AXP', 0.034, 0.03, -11.863],
 ['BA', 0.034, 0.046, 33.537],
 ['CAT', 0.034, 0.03, -11.863],
 ['CRM', 0.034, 0.04, 15.588],
 ['CSCO', 0.034, 0.03, -11.863],
 ['CVX', 0.034, 0.03, -11.62],
 ['DIS', 0.034, 0.03, -11.863],
 ['GS', 0.034, 0.044, 27.801],
 ['HD', 0.034, 0.03, -11.863],
 ['HON', 0.034, 0.054, 55.81],
 ['IBM', 0.034, 0.031, -8.682],
 ['INTC', 0.034, 0.03, -11.863],
 ['JNJ', 0.034, 0.03, -11.863],
 ['JPM', 0.034, 0.03, -11.863],
 ['KO', 0.034, 0.035, 1.544],
 ['MCD', 0.034, 0.03, -11.863],
 ['MMM', 0.034, 0.03, -11.863],
 ['MRK', 0.034, 0.03, -11.863],
 ['MSFT', 0.034, 0.03, -11.863],
 ['NKE', 0.034, 0.033, -4.444],
 ['PG', 0.034, 0.033, -4.735],
 ['TRV', 0.034, 0.03, -11.863],
 ['UNH', 0.034, 0.059, 71.64],
 ['V', 0.034, 0.03, -11.863],
 ['VZ', 0.034, 0.034, -2.079],
 ['WBA', 0.034, 0.03, -11.863],
 ['WMT', 0.034, 0.036, 3.279]]